# Uncertanty analysis

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.inspection import permutation_importance

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold 

from tqdm.notebook import tqdm
import plotly.express as px
import numpy as np
import pandas as pd
import os

os.chdir("/home/rooda/Dropbox/Patagonia/")

## Data 

In [ ]:
# read file from datasets_signature.ipynb
dataset = pd.read_csv("MS2 Results/dataset_hydro_signatures.csv", index_col = 0)

sources = ["Outline", "Climate", "Volume", "GCM", "SSP", "BCM"]
metrics = ["peak_water_year", "peak_magnitude", "frecuency", "rate_of_change", "duration"]

In [ ]:
variable_importance = []

for metric in tqdm(metrics):
    
    # only basins with at least one glacier
    data = dataset.loc[metric]
    data = data.dropna(axis = 1) 
    
    # categorical variables
    x = data[sources]
    x = OrdinalEncoder().fit_transform(x)
    
    for basin in tqdm(data.columns[6:], leave = False):
        
        y = data[basin].values # for each basib
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1, random_state=123)

        # random forest regression model
        rf = RandomForestRegressor(n_estimators = 10, random_state = 123, n_jobs = -1)
        rf.fit(x_train, y_train)

        # permutation importance
        variable_importance_basin = permutation_importance(rf, x_test, y_test, random_state=123, n_jobs=-1, 
                                        n_repeats=30, scoring = 'neg_root_mean_squared_error')
        
        variable_importance_basin = pd.DataFrame([variable_importance_basin.importances_mean], columns = sources)
        variable_importance_basin["ID"] = basin
        variable_importance_basin["Metric"] = metric
        variable_importance.append(variable_importance_basin)
    
variable_importance = pd.concat(variable_importance)
variable_importance = variable_importance[["ID", "Metric", "Outline", "Climate", "Volume", "GCM", "SSP", "BCM"]]
variable_importance.to_csv("MS2 Results/variable_importance.csv", index = False)